In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")

model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

In [4]:
x = "Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"

In [5]:
tokenizer(x)

{'input_ids': [101, 10930, 7743, 14855, 3627, 2003, 2062, 10514, 9468, 2229, 3993, 2059, 2017, 1005, 2222, 2412, 2022, 2054, 2015, 2039, 2007, 2017, 1998, 22650, 2017, 6517, 9587, 11263, 3600, 3022, 1012, 1012, 1012, 1045, 2323, 7743, 14308, 24471, 9004, 9072, 2594, 2317, 5344, 1998, 2131, 2017, 2000, 3610, 2026, 4632, 2017, 4364, 5305, 2368, 2033, 1012, 14855, 3627, 2003, 2055, 6620, 1999, 4830, 2189, 2158, 1012, 2123, 2102, 4487, 4757, 2008, 4485, 2006, 2032, 1012, 1998, 24218, 2003, 3308, 21388, 2078, 2066, 10722, 19498, 2002, 2001, 1037, 2567, 2205, 1012, 1012, 1012, 6616, 2378, 2317, 3337, 2131, 2477, 2157, 2279, 2051, 1012, 1010, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': 

In [ ]:
!pip install detoxify

In [ ]:
import pandas as pd
from detoxify import Detoxify
from tqdm import tqdm


import urllib.request
urllib.request.urlretrieve("https://drive.google.com/uc?id=1zUg1d5XqRRH_ddSz8MTd8KQuY8uNpDWB&export=download", "test.csv")

test_set = pd.read_csv('test.csv')

In [ ]:
test_set

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [ ]:
model = Detoxify('multilingual', device='cuda')

columns = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']


df = pd.DataFrame()

n = test_set.shape[0]

comments = list(test_set['comment_text'])
batch_size = 250

for i in tqdm(range(0,n,batch_size)):
  batch = comments[i:min(i+batch_size,n)]
  result = model.predict(batch)
  df = pd.concat([df, pd.DataFrame(result)]).reset_index(drop=True)
  

# Махинации с колонками
results = pd.DataFrame(df).drop('sexual_explicit', axis = 1)

results = results.rename(columns={'identity_attack'	: 'identity_hate', 'toxicity': 'toxic', 'severe_toxicity' : 'severe_toxic'})


results = results[columns]

df_merged = pd.concat([test_set['id'], results], axis=1)
df_merged.to_csv('result.csv',index=False)

100%|██████████| 613/613 [1:15:36<00:00,  7.40s/it]


In [ ]:
df_merged

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.994882,0.522357,0.983772,0.051912,0.977924,0.268910
1,0000247867823ef7,0.000294,0.000041,0.000202,0.000046,0.000271,0.000184
2,00013b17ad220c46,0.000290,0.000411,0.004054,0.000246,0.004396,0.000858
3,00017563c3f7919a,0.000330,0.000033,0.000098,0.000050,0.000124,0.000126
4,00017695ad8997eb,0.000671,0.000029,0.000116,0.000049,0.000247,0.000225
5,0001ea8717f6de06,0.000381,0.000036,0.000233,0.000046,0.000260,0.000107
6,00024115d4cbde0f,0.000731,0.000026,0.000151,0.000030,0.000442,0.000149
7,000247e83dcc1211,0.562206,0.001329,0.111584,0.000820,0.088132,0.011372
8,00025358d4737918,0.360191,0.000652,0.024870,0.000576,0.068838,0.001365
9,00026d1092fe71cc,0.000301,0.000028,0.000140,0.000035,0.000211,0.000117
